In [ ]:
#find the oil price in Khon Kaen
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
import time
from bs4 import BeautifulSoup as soup
import pandas as pd

driver = webdriver.Chrome(executable_path=r'') # put the Chrome driver path
url = 'https://www.pttor.com/th/oil_price'
driver.get(url) # open the website
button = driver.find_element(By.XPATH,'//*[@id="__layout"]/div/div/section/div[2]/div/div/div[1]/div[1]/div[1]/p')

time.sleep(3)
#Click on Provincial price
actionchain = ActionChains(driver)
ActionChains(driver).click(button).perform()

time.sleep(3)
province = Select(driver.find_element(By.XPATH, '//*[@id="__layout"]/div/div/section/div[2]/div/div/div[2]/form/div[1]/select'))
province.select_by_visible_text('ขอนแก่น')

time.sleep(3)
month = Select(driver.find_element(By.XPATH,'//*[@id="__layout"]/div/div/section/div[2]/div/div/div[2]/form/div[3]/select'))
month.select_by_visible_text('ทุกเดือน')

price_list = []
headers_list = ['Timestamp',
                'Diesel B20',
                'Diesel',
                'Diesel B7',
                'E85',
                'E20',                    
                'Gasohol 91',
                'Gasohol 95',
                'Benzene',
                'Superpower Diesel B7',
                'Superpower Gasohol 95']


for page in range(5):
    #Start Scraping
    time.sleep(3)
    page_html = driver.page_source

    data = soup(page_html,'html.parser') #scan the html
    table = data.findAll('table') #get the table element in the html

    body = table[0].findAll('tbody') #find the element that is tbody
    rows = body[0].findAll('tr') # find each tr in the tbody


    """
    Create the dictionary of the price list in a day.
    Using for loop to find the daily price that has already scraped from the tbody part.
    """

    for i in range(len(rows)):
        day_price = rows[i].findAll('td')
        day_price_list = []
        for oil in day_price:
            price = oil.text #convert into text 
            price = price.replace("  ","") #clean the data with 2 or upper white spaces (1 space will make the time stick with year)
            price = price.replace('\n',"") #in html there is \n, so we need to clean
            day_price_list.append(price) #create list of price with clean text

        price_dict = {}
            # create the price dict and let the headers be the Key and the time/price be the Values

        for head,price in zip(headers_list,day_price_list):
            price_dict[head] = price
        price_list.append(price_dict) # append to the price_list

        #Click on next page
    next_page = driver.find_element(By.XPATH,'//*[@id="__layout"]/div/div/section/div[2]/div/div/div[2]/div/div[2]/div/div/ul/li[5]/a')
    actionchain_2 = ActionChains(driver)
    ActionChains(driver).click(next_page).perform()
    time.sleep(15)

driver.close() # close when finished

#Make it into DataFrame
df = pd.DataFrame(price_list)
df.drop_duplicates(subset=['Timestamp'], inplace=True)
df.set_index('Timestamp', inplace = True) #set the index to Timestamp
df
df.to_csv('oil-price-2022.csv')